#### LoRA

LoRA, or Low-Rank Adaptation, is a technique designed for fine-tuning large models efficiently by only updating a small set of additional parameters. Instead of modifying all parameters of a pretrained model, LoRA keeps the original weights frozen and introduces two low-rank matrices that approximate the necessary changes during training.

```py
W_new = W_original + ΔW
ΔW = A × B
```

- W_original: Frozen pre-trained weights
- A: Matrix of size (d × r)
- B: Matrix of size (r × k)
- r: Rank (much smaller than d and k)

```py
Original: h = W × x
LoRA: h = W × x + (B × A) × x
```

This low-rank decomposition allows the model to adapt to new tasks with significantly fewer trainable parameters, reducing both memory usage and computational overhead during training.



In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig

# Load a small model (adjust the model name as needed)
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# Set tokenizer chat template if needed
if not tokenizer.chat_template:
    tokenizer.chat_template = """{% for message in messages %}
            {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
            {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
            {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
            {% endif %}
            {% endfor %}"""

if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Assume train_dataset is already loaded and pre-processed,
# for example, by using load_dataset from the datasets library.
# Here we select a small subset for quick demonstration.
USE_GPU = False
if not USE_GPU:
    train_dataset = train_dataset.select(range(100))

In [ ]:
# Define SFT configuration along with LoRA hyperparameters:
# Here, lora_r (rank) and lora_alpha (scaling factor) are added.
sft_config = SFTConfig(
    learning_rate=8e-5,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    logging_steps=2,
    lora_r=8,        # Low-rank dimension
    lora_alpha=16,   # Scaling factor for LoRA updates
)

# Create SFTTrainer with LoRA-enabled configuration
sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

# Start training
sft_trainer.train()